In [6]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [7]:
#read file
file = open('port5.txt', 'r')
stock_number = int(file.readline())
stock_mean = []
stock_std = []
stock_cov = []

for i in range(stock_number):
    temp = file.readline().split()
    temp = [float(k) for k in temp]
    stock_mean.append(temp[0])
    stock_std.append(temp[1])
    
for i in range(stock_number):
    temp_list = []
    for j in range(stock_number):
        if(j < i):
            #temp_list.append(stock_cov[j][i])
            temp_list.append(0)
        else:
            temp = file.readline().split()
            temp_list.append(float(temp[2]))
    stock_cov.append(temp_list)
    

data ={'stock':[i for i in range(stock_number)],
                   'mean':stock_mean,
                   'std':stock_std,
      'cov':stock_cov}
data = pd.DataFrame(data)
stock_cov = np.array(stock_cov)
file.close()

In [8]:
print(data)

     stock      mean       std  \
0        0 -0.001117  0.037894   
1        1  0.003123  0.049735   
2        2 -0.003471  0.048318   
3        3 -0.000585  0.043137   
4        4 -0.000769  0.035887   
..     ...       ...       ...   
220    220 -0.002030  0.036741   
221    221 -0.002433  0.042138   
222    222  0.000065  0.034573   
223    223  0.000060  0.038612   
224    224 -0.000992  0.028306   

                                                   cov  
0    [1.0, 0.400689, 0.533499, 0.575209, 0.524052, ...  
1    [0, 1.0, 0.394083, 0.29892, 0.376573, 0.336374...  
2    [0, 0, 1.0, 0.56979, 0.567901, 0.661618, 0.640...  
3    [0, 0, 0, 1.0, 0.522017, 0.523995, 0.529387, 0...  
4    [0, 0, 0, 0, 1.0, 0.601431, 0.545058, 0.574818...  
..                                                 ...  
220  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
221  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
222  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
223  [0, 0, 0, 0, 0, 

In [10]:
#Create an empty model
m = gp.Model('protfolio')

# add a variable for each stock
var = pd.Series(m.addVars([i for i in range(stock_number)]))

# set objective
obj = (var*data['std']).dot(stock_cov).transpose().dot(var*data['std'])

m.setObjective(obj, GRB.MINIMIZE)

# set constrain on budget
m.addConstr(var.sum() == 1, 'budget')

# set constrain on expect revenue
r = m.addVar(name = 'R')

m.addConstr(var.dot(data['mean']) - r == 0, 'expect_revenue')
m.update()
# optimize model to find the minimum risk portfolio
m.setParam('OutputFlag', 0)
m.params.NonConvex = 2
m.optimize()
#for i in var:
#    if i.X > 1e-6:
#        print(i.varName, i.X)

print("expected revenue = ",m.getVarByName('R').X)
print("min risk = ", m.getObjective().getValue())


expected revenue =  0.0002797591621579848
min risk =  0.00018675780942802318
